In [1]:
import pandas as pd

# Procesamiento de texto
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
entidades = pd.read_csv('../data/bronze/entidades_bronce.csv', sep=';')
print(entidades.shape)
print(entidades.columns)
entidades.head(2)

(12989, 6)
Index(['NOMBRE', 'CODIGO', 'NIT', 'ORDEN', 'DEPARTAMENTO', 'MUNICIPIO'], dtype='object')


,NOMBRE,CODIGO,NIT,ORDEN,DEPARTAMENTO,MUNICIPIO
0,BOYACÁ EMPRESA DEPARTAMENTAL DE SERVICIOS PÚB...,215001021.0,900297725,TERRITORIAL,BOYACÁ,TUNJA
1,CUNDINAMARCA ALCALDÍA MUNICIPIO DE LA VEGA,225402011.0,800128428-1,TERRITORIAL,CUNDINAMARCA,LA VEGA


In [3]:
stop_words = set(stopwords.words("spanish"))

def clean_text(text):
    text = text.lower()
    text = re.sub(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)', "", text)
    text = re.sub(r'[\\!\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~]', "", text)
    text = re.sub(r'\#\.', '', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'  ', ' ', text)
    text = re.sub(r'´', '',text)
    text = re.sub(r',', '',text)
    text = re.sub(r'\-', '', text)
    text = re.sub(r'á', 'a', text)
    text = re.sub(r'é', 'e', text)
    text = re.sub(r'í', 'i', text)
    text = re.sub(r'ó', 'o', text)
    text = re.sub(r'ú', 'u', text)
    text = re.sub(r'ò', 'o', text)
    text = re.sub(r'à', 'a', text)
    text = re.sub(r'è', 'e', text)
    text = re.sub(r'ì', 'i', text)
    text = re.sub(r'ù', 'u', text)
    text = re.sub("\d+", ' ', text)
    text = re.sub("\\s+", ' ', text)
    text = text.upper()
    tokens = word_tokenize(text)
    tokens = [w for w in tokens if not w.lower() in stop_words]
    tokens = " ".join(tokens)
    return tokens

In [4]:
entidades['NOMBRE'] = entidades['NOMBRE'].astype(str)
entidades['DEPARTAMENTO'] = entidades['DEPARTAMENTO'].astype(str)
entidades['MUNICIPIO'] = entidades['MUNICIPIO'].astype(str)

entidades['NOMBRE'] = entidades['NOMBRE'].apply(clean_text)
entidades['DEPARTAMENTO'] = entidades['DEPARTAMENTO'].apply(clean_text)
entidades['MUNICIPIO'] = entidades['MUNICIPIO'].apply(clean_text)

In [5]:
entidades = entidades.drop_duplicates()
entidades.shape

(12989, 6)

In [6]:
entidades['NIT'] = entidades['NIT'].astype(str)
entidades['NIT'] = entidades['NIT'].str.split('-').str[0]

In [7]:
entidades = entidades.drop_duplicates()
entidades.shape

(12989, 6)

In [8]:
entidades[entidades['DEPARTAMENTO'].str.contains('BOGO')]['DEPARTAMENTO'].value_counts()

DEPARTAMENTO
BOGOTA D.C .               584
DISTRITO CAPITAL BOGOTA    307
Name: count, dtype: int64

In [9]:
entidades.loc[entidades['DEPARTAMENTO'].str.contains('BOGO'), 'DEPARTAMENTO'] = 'BOGOTA D.C .'
entidades.loc[entidades['MUNICIPIO'].str.contains('BOGO'), 'MUNICIPIO'] = 'BOGOTA D.C .'
entidades = entidades.drop_duplicates()
entidades.shape

(12989, 6)

In [10]:
entidades.head()

,NOMBRE,CODIGO,NIT,ORDEN,DEPARTAMENTO,MUNICIPIO
0,BOYACA EMPRESA DEPARTAMENTAL SERVICIOS PUBLICO...,215001021.0,900297725,TERRITORIAL,BOYACA,TUNJA
1,CUNDINAMARCA ALCALDIA MUNICIPIO VEGA,225402011.0,800128428,TERRITORIAL,CUNDINAMARCA,VEGA
2,GUAJIRA ADMINISTRACION TEMPORAL SECTOR AGUA PO...,244001019.0,892115015,TERRITORIAL,GUAJIRA,RIOHACHA
3,CAR CORPORACION AUTONOMA REGIONAL CUNDINAMARCA,132003003.0,899999062,TERRITORIAL,BOGOTA D.C .,BOGOTA D.C .
4,ANTIOQUIA ALCALDIA MUNICIPIO COPACABANA,205212011.0,nan,TERRITORIAL,ANTIOQUIA,COPACABANA


In [11]:
entidades.to_csv('../data/silver/entidades_silver.csv', sep=';', index=False)